# Project for DNDS course at CEU

In my project I try to use data mining, Mashine Learning and NLP techniques for analysis of procurement contracts in Russia. My project is split into several parts. 

# Part 1 - Data Collection

The data on procurement contracts lies on ftp server of the website zakupki.gov ru.It is a big data set, so I just use a small sample of the flies. I use Moscow as a sample region. In Moscow I focuse on folders with notifications and contracts for electronic auction. Notifications - are initial documents which are published when auction is about to be held. It contains description of the product as well as the maximum price the buyer is willing to pay (that the starting price of electronic auction). The contract contains the results of the auction,number of participants, execution time etc. I downloaded data in this notebook for protocols as well (they contain changes in the contract), but in the end in this project I didn't use it.

I start with collecting data from ftp server. Due to situation in Russia, access to government websites (including procurement) was blocked from other countries. I was able to find out that it is still possible to access this website from Austria if I use VPN (I used NordVPN) and pretended to be in India (other countries didn't work). So, most likely to run this code you will need to use VPN as well. Moreover, some parts of the code take many hours to run (up to 8), so be aware of this. Unfortunately, for some of the parts of the code, I don't provide output in jupyter notebook after running simply because it was too long to rerun it if already did it. 

In [44]:
# Import libraries to work with ftp server 
import ftplib
from ftplib import FTP
import os
import pandas as pd

In [45]:
# The data lies on the FTP server of the Russian procurement website (zakupki.gov.ru)
# There are 2 different parts on server for 2 Federal Laws (FL) under which procurement in Russia is made
# I am interested in data under FL 44 and access its FTP server part
ftp = FTP('ftp.zakupki.gov.ru')
ftp.login(user='free', passwd='free')

'230 Login successful.'

In [3]:
# Retrlines allows me to pass comands to ftp. Here I want to see a list of folders on this ftp server
data = ftp.retrlines('LIST')
print(data)

drwxr-xr-x   92 1040     1041         8192 Mar 01 22:00 94fz
-r--r-xr-x    1 1039     1041          713 Jan 04  2014 _readme.txt
drwxrwx---    2 1092     1100         8192 Sep 24  2014 customer_verification
drwxrwxrwx    2 1109     1040            6 Nov 15  2019 fcs_SFM
drwxrwxr-x  747 1080     1041       454656 Jul 02  2018 fcs_banks
drwxr-xr-x    2 1151     1158           62 Jul 13  2018 fcs_banks_hidden
drwxr-xr-x    9 1094     1042        12288 Apr 04  2020 fcs_discussion
drwxr-xr-x   11 1093     1040         8192 Apr 12  2018 fcs_fas
drwxr-xr-x   98 1079     1040         4096 Jan 28 18:54 fcs_nsi
drwxr-xr-x   95 1077     1040     21737472 Mar 22 11:18 fcs_regions
drwxr-xr-x   28 1078     1041         4096 Mar 03 17:01 fcs_rules
drwxr-xr-x    3 1097     1043           30 Jul 28  2015 fcs_sc
drwxrwx---    2 1089     1097          206 Mar 11  2016 offline_rw
drwxrwx---    3 1219     1226        69632 Mar 24 18:05 rnp
-rw-r--r--    1 0        0               0 Feb 20  2017 тест.txt
22

In [4]:
# Let's look one step futher into the folder for regions
ftp.cwd('fcs_regions') #Learning Outcome 1 (LO1 - Understand folder structure. Perform operations on files in different folders)
data = ftp.retrlines('LIST')
print(data)

-rw-r--r--    1 0        0               0 Apr 19  2021 1.txt
-rwxr-xr-x    1 0        0           11484 Apr 19  2021 1214.sh
-rw-r--r--    1 0        0           10346 Mar 31  2021 23.txt
drwxr-xr-x   21 1077     1040         4096 Apr 02  2021 Adygeja_Resp
drwxr-xr-x   22 1077     1040         4096 Apr 19  2021 Altaj_Resp
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Altajskij_kraj
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Amurskaja_obl
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Arkhangelskaja_obl
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Astrakhanskaja_obl
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Bajkonur_g
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Bashkortostan_Resp
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Belgorodskaja_obl
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Brjanskaja_obl
drwxr-xr-x   22 1077     1040         4096 Apr 02  2021 Burjatija_Resp
drwxr-xr-x   22 1077     1

In [13]:
# And now let's look specifically at folder for Moscow ( I will only have memory for one region and Moscow is the largest)
ftp.cwd('Moskva')
data = ftp.retrlines('LIST')
print(data)

drwxr-xr-x    4 1077     1040         8192 Mar 05 01:08 addinfo
drwxr-xr-x    4 1077     1040         8192 Mar 05 01:07 auditresult
drwxr-xr-x    4 1077     1040       126976 Mar 05 21:14 contractprojects
drwxr-xr-x    4 1077     1040       958464 Mar 06 23:11 contracts
drwxr-xr-x    9 1077     1040       114688 Apr 09  2021 control99docs
drwxr-xr-x    4 1077     1040        77824 Mar 05 03:00 customerreports
drwxr-xr-x    4 1077     1040         8192 Mar 05 01:53 notificationExceptions
drwxr-xr-x    4 1077     1040       376832 Mar 05 18:47 notifications
drwxr-xr-x    4 1077     1040        24576 Mar 05 17:21 plangraphs
drwxr-xr-x    4 1077     1040        36864 Oct 02 12:20 plangraphs2017
drwxr-xr-x    4 1077     1040        45056 Mar 05 23:28 plangraphs2020
drwxr-xr-x    7 1077     1040          101 Apr 12  2018 pprf615docs
drwxr-xr-x    4 1077     1040       217088 Mar 05 18:44 protocols
drwxr-xr-x    4 1077     1040          128 Jun 02  2021 protocolsfrequnl
drwxr-xr-x    4 1077  

In [6]:
# And inside Moscow I need notifications, protocols and contracts. First I look into notifications
ftp.cwd('notifications')
data = ftp.retrlines('LIST')
print(data)

drwxr-xr-x    2 1077     1040         8192 Mar 24 00:32 currMonth
-rw-rw-r--    1 1077     1040      2534570 May 13  2014 notification_Moskva_2014010100_2014020100_086.xml.zip
-rw-rw-r--    1 1077     1040       474101 May 13  2014 notification_Moskva_2014010100_2014020100_087.xml.zip
-rw-rw-r--    1 1077     1040         6289 May 13  2014 notification_Moskva_2014010100_2014020100_088.xml.zip
-rw-rw-r--    1 1077     1040         4620 Mar 20  2014 notification_Moskva_2014010600_2014010700_001.xml.zip
-rw-rw-r--    1 1077     1040         8722 Mar 20  2014 notification_Moskva_2014010900_2014011000_001.xml.zip
-rw-rw-r--    1 1077     1040        28152 Mar 20  2014 notification_Moskva_2014011000_2014011100_001.xml.zip
-rw-rw-r--    1 1077     1040        21094 Mar 20  2014 notification_Moskva_2014011100_2014011200_001.xml.zip
-rw-rw-r--    1 1077     1040        25841 Mar 21  2014 notification_Moskva_2014011300_2014011400_001.xml.zip
-rw-rw-r--    1 1077     1040       102046 Mar 21  201

In [7]:
out = "C:/Users/Имя/termpaper/notifications"

In [8]:
#I use ftp.nlst command to get names of files and catalogs in this folder
filenames = reversed(ftp.nlst())

In [9]:
for f in filenames:
    print(f)

prevMonth
notification_Moskva_2022020100_2022030100_019.xml.zip
notification_Moskva_2022020100_2022030100_018.xml.zip
notification_Moskva_2022020100_2022030100_017.xml.zip
notification_Moskva_2022020100_2022030100_016.xml.zip
notification_Moskva_2022020100_2022030100_015.xml.zip
notification_Moskva_2022020100_2022030100_014.xml.zip
notification_Moskva_2022020100_2022030100_013.xml.zip
notification_Moskva_2022020100_2022030100_012.xml.zip
notification_Moskva_2022020100_2022030100_011.xml.zip
notification_Moskva_2022020100_2022030100_010.xml.zip
notification_Moskva_2022020100_2022030100_009.xml.zip
notification_Moskva_2022020100_2022030100_008.xml.zip
notification_Moskva_2022020100_2022030100_007.xml.zip
notification_Moskva_2022020100_2022030100_006.xml.zip
notification_Moskva_2022020100_2022030100_005.xml.zip
notification_Moskva_2022020100_2022030100_004.xml.zip
notification_Moskva_2022020100_2022030100_003.xml.zip
notification_Moskva_2022020100_2022030100_002.xml.zip
notification_Moskv

In [10]:
filenames = reversed(ftp.nlst())

In [11]:
#Now I download files from "notifications" folder
for filename in filenames:
    if os.path.exists("C:/Users/Имя/termpaper/notifications/" + filename) == False:
        host_file = os.path.join (out, filename)
        try:
            with open(host_file, 'wb') as local_file:
                ftp.retrbinary('RETR ' + filename, local_file.write)
        except ftplib.error_perm:
            pass
    elif os.path.exists(("C:/Users/Имя/termpaper/notifications/" + filename)) == True:
            print('File ', filename, ' Already Exists, Skipping Download')
 
ftp.quit()

File  prevMonth  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_019.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_018.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_017.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_016.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_015.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_014.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_013.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_012.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_011.xml.zip  Already Exists, Skipping Download
File  notification_Moskva_2022020100_2022030100_010.xml.zip  Already Exists, Skipping Download

KeyboardInterrupt: 

In [46]:
# But I also need files from "contracts" folder. So I repeat all the same things but for "contracts"
#First I get the new folder I need on ftp server
ftp = FTP('ftp.zakupki.gov.ru')
ftp.login(user='free', passwd='free')
ftp.cwd('fcs_regions')
ftp.cwd('Moskva')
ftp.cwd('contracts')



'250 Directory successfully changed.'

In [47]:
os.getcwd()

'C:\\Users\\Имя\\Downloads\\termpaper-main\\termpaper-main'

In [48]:
#I check which files are inside
#filenames = ftp.nlst()
filenames = reversed(ftp.nlst())


In [49]:
#I download files from "contacts" folder
out = "C:/Users/Имя/termpaper/contracts"
for filename in filenames:
    if os.path.exists("C:/Users/Имя/termpaper/contracts/" + filename) == False:
        host_file = os.path.join (out, filename)
        try:
            with open(host_file, 'wb') as local_file:
                ftp.retrbinary('RETR ' + filename, local_file.write)
        except ftplib.error_perm:
            pass
    elif os.path.exists(("C:/Users/Имя/termpaper/contracts/" + filename)) == True:
            print('File ', filename, ' Already Exists, Skipping Download')
 
ftp.quit()

File  prevMonth  Already Exists, Skipping Download
File  currMonth  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_180.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_179.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_178.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_177.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_176.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_175.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_174.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_173.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_172.xml.zip  Already Exists, Skipping Download
File  contract_Moskva_2022020100_2022030100_171.xml.zip  Already Exists, Skippi

KeyboardInterrupt: 

In [ ]:
# But I also need files from "protocols" folder. So I repeat all the same things but for "protocols"
#First I get the new folder I need on ftp server
ftp = FTP('ftp.zakupki.gov.ru')
ftp.login(user='free', passwd='free')
ftp.cwd('fcs_regions')
ftp.cwd('Moskva')
ftp.cwd('protocols')

In [ ]:
#I check which files are inside
filenames = ftp.nlst()
print(filenames)

In [ ]:
filenames = reversed(ftp.nlst())

In [ ]:
#I download files from "protocols" folder
out = "C:/Users/Имя/termpaper/protocols"
for filename in filenames:
    if os.path.exists("C:/Users/Имя/termpaper/protocols/" + filename) == False:
        host_file = os.path.join (out, filename)
        try:
            with open(host_file, 'wb') as local_file:
                ftp.retrbinary('RETR ' + filename, local_file.write)
        except ftplib.error_perm:
            pass
    elif os.path.exists(("C:/Users/Имя/termpaper/protocols/" + filename)) == True:
            print('File ', filename, ' Already Exists, Skipping Download')
 
ftp.quit()

# Part 2: Unzipping files, parsing relevant parts of xml files and convertion to csv format

I see that downloaded folders are xml and sig files in zipped in many folders. My strategy is to unzip files, limit them
only to xml files and to files relevant for electronic auctions (EA44) where it possible. My research question this time is only about electronic auctions

The solution I found for bigger folders of is unzip based on condition and at least get only relevant xml files.I manually splitted downloaded zip files into smaller folders by year to have a smaller sample for the project. The important thing that for different years the name for electronic auctions may be different, so I pay attention to it.


In [7]:
#install all libraries
import csv
import xml.etree.ElementTree as ET
import os
from bs4 import BeautifulSoup

In [11]:
#First I want to extract notifications for electronic auctions for 2021, there electronic auction is defined as "fksNotificationEA44"
#The important thing is that there are no 0 volume files, otherwise it won't work, but it is not the case for me
import os, zipfile

dir_name = "C:/Users/Имя/termpaper/notifications_2021"
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        listOfFileNames = zip_ref.namelist()
        for fileName in listOfFileNames:
            if fileName.endswith('.xml'):
                if fileName.startswith('fksNotificationEA44'):
                    zip_ref.extract(fileName)

In [ ]:
#Now I want to do same for notifications 2020
import os, zipfile
dir_name = "C:/Users/Имя/termpaper/notifications_2020"
extension = ".zip"

os.chdir(dir_name)

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        listOfFileNames = zip_ref.namelist()
        for fileName in listOfFileNames:
            if fileName.endswith('.xml'): #i choose files only with xml extension, because there are also sig files
                if fileName.startswith('fksNotificationEA44'): # code for electronic auction notification
                    zip_ref.extract(fileName)
    

In [ ]:
#Now I want to do same for notifications 2022, but name for electronic auctions is different
dir_name = "C:/Users/Имя/termpaper/notifications_2022"
extension = ".zip"

os.chdir(dir_name)

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        listOfFileNames = zip_ref.namelist()
        for fileName in listOfFileNames:
            if fileName.endswith('.xml'): #i choose files only with xml extension, because there are also sig files
                if fileName.startswith('epNotificationEF'): # code for electronic auction notification
                    zip_ref.extract(fileName)

In [1]:
# Now I want to run the same for contracts. I first extract the data for 2022 contracts
import os, zipfile

dir_name = "C:/Users/Имя/termpaper/contracts"
extension = ".zip"

In [2]:
os.getcwd()

'C:\\Users\\Имя\\Downloads\\termpaper-main\\termpaper-main'

In [55]:
import subprocess

In [3]:
os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        try:
            zip_ref = zipfile.ZipFile(file_name)
            listOfFileNames = zip_ref.namelist()
            for fileName in listOfFileNames:
                if fileName.endswith('.xml'):
                    if fileName.startswith('contract_'):
                        zip_ref.extract(fileName)
        except: pass
            

# Parsing xml files 

First, I want to take one file as example and try to parse it to understand its structure and which parts of the file I need. I also upload notification and contract example on github, so you can try it as example

In [4]:
#change directory
os.chdir("C:/Users/Имя/termpaper/examples")

In [10]:
#I want to see the structure of the file in the notification as an example

with open('xml_notification_example.xml', encoding ='utf-8') as f:
    soup = BeautifulSoup(f, 'xml')
print(soup)

<?xml version="1.0" encoding="utf-8"?>
<ns2:export xmlns="http://zakupki.gov.ru/oos/types/1" xmlns:ns10="http://zakupki.gov.ru/oos/printform/1" xmlns:ns11="http://zakupki.gov.ru/oos/control99/1" xmlns:ns2="http://zakupki.gov.ru/oos/export/1" xmlns:ns3="http://zakupki.gov.ru/oos/common/1" xmlns:ns4="http://zakupki.gov.ru/oos/base/1" xmlns:ns5="http://zakupki.gov.ru/oos/TPtypes/1" xmlns:ns6="http://zakupki.gov.ru/oos/CPtypes/1" xmlns:ns7="http://zakupki.gov.ru/oos/pprf615types/1" xmlns:ns8="http://zakupki.gov.ru/oos/EPtypes/1" xmlns:ns9="http://zakupki.gov.ru/oos/SMTypes/1">
<ns2:fcsNotificationEF schemeVersion="11.0">
<id>25499274</id>
<purchaseNumber>0101200007320000078</purchaseNumber>
<docPublishDate>2021-01-12T15:23:29.572+03:00</docPublishDate>
<href>https://zakupki.gov.ru/epz/order/notice/ea44/view/common-info.html?regNumber=0101200007320000078</href>
<printForm>
<url>https://zakupki.gov.ru/epz/order/notice/printForm/view.html?regNumber=0101200007320000078</url>
</printForm>
<purc

In [11]:
# Now I want to prepare for parsing this example files 
tree = ET.parse("xml_notification_example.xml")
print(tree)

In [12]:
root = tree.getroot()
print(root)

<Element '{http://zakupki.gov.ru/oos/export/1}export' at 0x000002B9CAFB8810>


In [13]:
# I want to decode xml file to convert it to csv later. I do it for electronic auctions notifications. Although there is 
#a lot of information in one xml file, I only extract relevant information for my research question.

# ! Ideally,I also need information about supplier and group to which procurement objects belong (based on OKPD codes),
# but there may be several of them for one purchase, so later i need to think how to include this as well

# Although the following code worked for 1 example xml file, there should be some minor changes if I want to loop over
#several files and later I implemented these changes.


xml_data_to_csv = open('out.csv', 'w',encoding = 'utf-8')
csv_writer = csv.writer(xml_data_to_csv)

list_head = []
list_nodes = []
    

list_head.append('id')
list_head.append('purchasenumber')
list_head.append('docpublishdate')
list_head.append('href')
list_head.append('printform_url')
list_head.append('purchaseobjectinfo')
list_head.append('responsibleorg_regnum')
list_head.append('responsibleorg_fullname')
list_head.append('responsibleorg_postaddress')
list_head.append('responsibleorg_factaddress')
list_head.append('responsibleorg_inn')
list_head.append('responsibleorg_kpp')
list_head.append('responsiblerole')
list_head.append('placingway_code')
list_head.append('placingway_name')
list_head.append('collecting_startdate')
list_head.append('collecting_place')
list_head.append('collecting_order')
list_head.append('collecting_enddate')
list_head.append('scoring_date')
list_head.append('bidding_date')
list_head.append('max_price')
list_head.append('currency_code')
list_head.append('currency_name')
list_head.append('finance_source')
list_head.append('quantity_undefined')
list_head.append('customer_regnum')
list_head.append('customer_fullname')
list_head.append('platform_code')


csv_writer.writerow(list_head)


for element in root:
    iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
    list_nodes.append(iden)
    
    purchaseNumber = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseNumber').text
    list_nodes.append(purchaseNumber)
       
    docPublishDate = element.find('{http://zakupki.gov.ru/oos/types/1}docPublishDate').text
    list_nodes.append(docPublishDate)
    
    href = element.find('{http://zakupki.gov.ru/oos/types/1}href').text
    list_nodes.append(href)

    ns_1= element.find('{http://zakupki.gov.ru/oos/types/1}printForm')
    printform_url = ns_1.find('{http://zakupki.gov.ru/oos/types/1}url')
    list_nodes.append(printform_url.text)

    purchaseObjectInfo = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseObjectInfo').text
    list_nodes.append(purchaseObjectInfo)
    
    ns_2= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    ns_2_1 = ns_2.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
    responsibleorg_regnum = ns_2_1.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
    list_nodes.append(responsibleorg_regnum)
    
    ns_3= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    ns_3_1 = ns_3.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
    responsibleorg_fullname = ns_3_1.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
    list_nodes.append(responsibleorg_fullname)
    
    ns_4= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    ns_4_1 = ns_4.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
    responsibleorg_postadress = ns_4_1.find('{http://zakupki.gov.ru/oos/types/1}postAddress').text
    list_nodes.append(responsibleorg_postadress)
    
    ns_5= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    ns_5_1 = ns_5.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
    responsibleorg_factaddress = ns_5_1.find('{http://zakupki.gov.ru/oos/types/1}factAddress').text
    list_nodes.append(responsibleorg_factaddress)

    ns_6= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    ns_6_1 = ns_6.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
    responsibleorg_inn = ns_6_1.find('{http://zakupki.gov.ru/oos/types/1}INN').text
    list_nodes.append(responsibleorg_inn)
    
    ns_7= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    ns_7_1 = ns_7.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
    responsibleorg_kpp = ns_7_1.find('{http://zakupki.gov.ru/oos/types/1}KPP').text
    list_nodes.append(responsibleorg_kpp)

    ns_8= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
    responsiblerole = ns_8.find('{http://zakupki.gov.ru/oos/types/1}responsibleRole').text
    list_nodes.append(responsiblerole)
    
    ns_9= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
    placingway_code = ns_9.find('{http://zakupki.gov.ru/oos/types/1}code').text
    list_nodes.append(placingway_code)
    
    ns_10= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
    placingway_name = ns_10.find('{http://zakupki.gov.ru/oos/types/1}name').text
    list_nodes.append(placingway_name)
    
    ns_11= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
    ns_11_1 = ns_11.find('{http://zakupki.gov.ru/oos/types/1}collecting')
    collecting_startdate = ns_11_1.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
    list_nodes.append(collecting_startdate)
    
    ns_12= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
    ns_12_1 = ns_12.find('{http://zakupki.gov.ru/oos/types/1}collecting')
    collecting_place = ns_12_1.find('{http://zakupki.gov.ru/oos/types/1}place').text
    list_nodes.append(collecting_place)
    
    ns_13= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
    ns_13_1 = ns_13.find('{http://zakupki.gov.ru/oos/types/1}collecting')
    collecting_order = ns_13_1.find('{http://zakupki.gov.ru/oos/types/1}order').text
    list_nodes.append(collecting_order)
    
    ns_14= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
    ns_14_1 = ns_14.find('{http://zakupki.gov.ru/oos/types/1}collecting')
    collecting_enddate = ns_14_1.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
    list_nodes.append(collecting_enddate)
    
    ns_15= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
    ns_15_1 = ns_15.find('{http://zakupki.gov.ru/oos/types/1}scoring')
    scoring_date = ns_15_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
    list_nodes.append(scoring_date)
    
    ns_16= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
    ns_16_1 = ns_16.find('{http://zakupki.gov.ru/oos/types/1}bidding')
    bidding_date = ns_16_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
    list_nodes.append(bidding_date)
    
    ns_17= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
    max_price = ns_17.find('{http://zakupki.gov.ru/oos/types/1}maxPrice').text
    list_nodes.append(max_price)
    
    ns_18= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
    ns_18_1 = ns_18.find('{http://zakupki.gov.ru/oos/types/1}currency')
    currency_code = ns_18_1.find('{http://zakupki.gov.ru/oos/types/1}code').text
    list_nodes.append(currency_code)
    
    ns_19= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
    ns_19_1 = ns_19.find('{http://zakupki.gov.ru/oos/types/1}currency')
    currency_name = ns_19_1.find('{http://zakupki.gov.ru/oos/types/1}name').text
    list_nodes.append(currency_name)
    
    try: #not all objects have finance source
        ns_20= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        finance_source = ns_20.find('{http://zakupki.gov.ru/oos/types/1}financeSource').text
        list_nodes.append(finance_source)
    except: 
        continue
    
    ns_21= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
    quantity_undefined = ns_21.find('{http://zakupki.gov.ru/oos/types/1}quantityUndefined').text
    list_nodes.append(quantity_undefined)
    
    ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
    ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
    ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
    ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
    customer_regnum = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
    list_nodes.append(customer_regnum)
    
    ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
    ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
    ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
    ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
    customer_fullname = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
    list_nodes.append(customer_fullname)
    

    ns23 = element.find('{http://zakupki.gov.ru/oos/types/1}ETP')
    platform_code = ns23.find('{http://zakupki.gov.ru/oos/types/1}code').text
    list_nodes.append(platform_code)
    
    
    csv_writer.writerow(list_nodes)
    

xml_data_to_csv.close()
     



In [14]:
print(list_head)
print(list_nodes)

['id', 'purchasenumber', 'docpublishdate', 'href', 'printform_url', 'purchaseobjectinfo', 'responsibleorg_regnum', 'responsibleorg_fullname', 'responsibleorg_postaddress', 'responsibleorg_factaddress', 'responsibleorg_inn', 'responsibleorg_kpp', 'responsiblerole', 'placingway_code', 'placingway_name', 'collecting_startdate', 'collecting_place', 'collecting_order', 'collecting_enddate', 'scoring_date', 'bidding_date', 'max_price', 'currency_code', 'currency_name', 'finance_source', 'quantity_undefined', 'customer_regnum', 'customer_fullname', 'platform_code']
['25499274', '0101200007320000078', '2021-01-12T15:23:29.572+03:00', 'https://zakupki.gov.ru/epz/order/notice/ea44/view/common-info.html?regNumber=0101200007320000078', 'https://zakupki.gov.ru/epz/order/notice/printForm/view.html?regNumber=0101200007320000078', 'Оказание услуг по техническому обслуживанию и текущему ремонту автомобилей', '01012000073', 'ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКОРТОСТАН ПРИ ПРЕЗИДЕНТЕ РОССИЙСКОЙ

In [17]:
#Next I want to decode contracts. There I am mostly interested in purchase number (a different name then in notifications),
#final price and execution time 
#I want to see the structure of the file which I use an example
#change directory
os.chdir("C:/Users/Имя/termpaper/examples")

with open('xml_contract_example.xml', encoding ='utf-8') as f:
    soup = BeautifulSoup(f, 'xml')
print(soup)

<?xml version="1.0" encoding="utf-8"?>
<ns2:export xmlns="http://zakupki.gov.ru/oos/types/1" xmlns:ns10="http://zakupki.gov.ru/oos/SMTypes/1" xmlns:ns11="http://zakupki.gov.ru/oos/printform/1" xmlns:ns12="http://zakupki.gov.ru/oos/control99/1" xmlns:ns2="http://zakupki.gov.ru/oos/export/1" xmlns:ns3="http://zakupki.gov.ru/oos/common/1" xmlns:ns4="http://zakupki.gov.ru/oos/base/1" xmlns:ns5="http://zakupki.gov.ru/oos/TPtypes/1" xmlns:ns6="http://zakupki.gov.ru/oos/KOTypes/1" xmlns:ns7="http://zakupki.gov.ru/oos/CPtypes/1" xmlns:ns8="http://zakupki.gov.ru/oos/pprf615types/1" xmlns:ns9="http://zakupki.gov.ru/oos/EPtypes/1">
<ns2:contract schemeVersion="12.0">
<id>70163711</id>
<placementDate>2014-11-24T15:30:22</placementDate>
<publishDate>2022-01-13T00:14:01.259+03:00</publishDate>
<versionNumber>25</versionNumber>
<foundation>
<fcsOrder>
<order>
<notificationNumber>0373100013114000083</notificationNumber>
<lotNumber>1</lotNumber>
<placing>12011</placing>
</order>
</fcsOrder>
</foundatio

In [16]:
tree = ET.parse("xml_contract_example.xml")
print(tree)
root = tree.getroot()
print(root)

<Element '{http://zakupki.gov.ru/oos/export/1}export' at 0x000002B9CADA9540>


In [18]:
#Same as before
# Changes are implemented when i later usei it to loop over files (for instance, I use later  try and except)
xml_data_to_csv = open('out_try.csv', 'w', encoding = 'utf-8')
csv_writer = csv.writer(xml_data_to_csv)

list_head = []
list_nodes = []
    

list_head.append('id_contract')
list_head.append('publishdate_contract')
list_head.append('purchasenumber')
list_head.append('final_price')
list_head.append('execution_startdate')
list_head.append('execution_enddate')
list_head.append('supplier_inn')

csv_writer.writerow(list_head)

for element in root:
        
        
        
        try:
            iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
            list_nodes.append(iden)
        except: continue
        
        try:
            publishdate_contract = element.find('{http://zakupki.gov.ru/oos/types/1}publishDate').text
            list_nodes.append(publishdate_contract)
        except: continue
        
        try:
            ns = element.find('{http://zakupki.gov.ru/oos/types/1}foundation')
            ns_1 = ns.find('{http://zakupki.gov.ru/oos/types/1}fcsOrder')
            ns_2 = ns_1.find('{http://zakupki.gov.ru/oos/types/1}order')
            purchasenumber = ns_2.find('{http://zakupki.gov.ru/oos/types/1}notificationNumber').text
            list_nodes.append(purchasenumber)
        except: continue
            
            
        try:
            ns1 = element.find('{http://zakupki.gov.ru/oos/types/1}priceInfo')
            final_price = ns1.find('{http://zakupki.gov.ru/oos/types/1}price').text
            list_nodes.append(final_price)
        except: continue
            
        try:
            ns2 = element.find('{http://zakupki.gov.ru/oos/types/1}executionPeriod')
            execution_startdate = ns2.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
            list_nodes.append(execution_startdate)
        except: continue
            
            
        try:
            ns3 = element.find('{http://zakupki.gov.ru/oos/types/1}executionPeriod')
            execution_enddate = ns3.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
            list_nodes.append(execution_enddate)
        except: continue
            
        try:
            ns4 = element.find('{http://zakupki.gov.ru/oos/types/1}suppliers')
            ns4_1 = ns4.find('{http://zakupki.gov.ru/oos/types/1}supplier')
            ns4_2 = ns4_1.find('{http://zakupki.gov.ru/oos/types/1}legalEntityRF')
            
            supplier_inn = ns4_2.find('{http://zakupki.gov.ru/oos/types/1}INN').text
            list_nodes.append(supplier_inn)
        except: continue    
    
csv_writer.writerow(list_nodes)
    
    
xml_data_to_csv.close()  


# Parsing over all files in the folder

That is the code for notifications 2021. For notifications 2022 it changed a bit

In [62]:
# Now I actually need to extrapolate this for the case when I have multiple xml files. They were in zip folders


import os
from xml.etree import ElementTree as ET
import csv


path = "C:/Users/Имя/termpaper/notifications_2021"

xml_data_to_csv = open('out_notifications_2021_ex.csv', 'w', encoding ='utf-8')
csv_writer = csv.writer(xml_data_to_csv)
    


list_head = []

    

list_head.append('id')
list_head.append('purchasenumber')
list_head.append('docpublishdate')
list_head.append('href')
list_head.append('printform_url')
list_head.append('purchaseobjectinfo')
list_head.append('responsibleorg_regnum')
list_head.append('responsibleorg_fullname')
list_head.append('responsibleorg_postaddress')
list_head.append('responsibleorg_factaddress')
list_head.append('responsibleorg_inn')
list_head.append('responsibleorg_kpp')
list_head.append('responsiblerole')
list_head.append('placingway_code')
list_head.append('placingway_name')
list_head.append('collecting_startdate')
list_head.append('collecting_place')
list_head.append('collecting_order')
list_head.append('collecting_enddate')
list_head.append('scoring_date')
list_head.append('bidding_date')
list_head.append('max_price')
list_head.append('currency_code')
list_head.append('currency_name')
list_head.append('finance_source')
list_head.append('quantity_undefined')
list_head.append('customer_regnum')
list_head.append('customer_fullname')
list_head.append('platform_code')

csv_writer.writerow(list_head)

for filename in os.listdir(path):
    
    # I want to get only xml files since I also have "sig" files
    if not filename.endswith('.xml'): continue
    # And only get files that start with 'fksNotificationEA44' since these are notifications for electronic auctions
    if not filename.startswith('fksNotificationEA44'): continue
    #for 2022 year  of notification it will be the following
    #if not filename.startswith('epNotificationEF2020_'): continue
    
    # And now let's parse these files and convert to csv as I did before for just 1 file:
    
    fullname = os.path.join(path, filename)
    
    tree = ET.parse(fullname)
    root = tree.getroot()
    
    for element in root:
        
        list_nodes= []
        
        iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
        list_nodes.append(iden)
    
        purchaseNumber = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseNumber').text
        list_nodes.append(purchaseNumber)
       
        docPublishDate = element.find('{http://zakupki.gov.ru/oos/types/1}docPublishDate').text
        list_nodes.append(docPublishDate)
    
        href = element.find('{http://zakupki.gov.ru/oos/types/1}href').text
        list_nodes.append(href)

        ns_1= element.find('{http://zakupki.gov.ru/oos/types/1}printForm')
        printform_url = ns_1.find('{http://zakupki.gov.ru/oos/types/1}url')
        list_nodes.append(printform_url.text)

        purchaseObjectInfo = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseObjectInfo').text
        list_nodes.append(purchaseObjectInfo)
    
        ns_2= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        ns_2_1 = ns_2.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
        responsibleorg_regnum = ns_2_1.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
        list_nodes.append(responsibleorg_regnum)
    
        ns_3= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        ns_3_1 = ns_3.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
        responsibleorg_fullname = ns_3_1.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
        list_nodes.append(responsibleorg_fullname)
    
        ns_4= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        ns_4_1 = ns_4.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
        responsibleorg_postadress = ns_4_1.find('{http://zakupki.gov.ru/oos/types/1}postAddress').text
        list_nodes.append(responsibleorg_postadress)
    
        ns_5= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        ns_5_1 = ns_5.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
        responsibleorg_factaddress = ns_5_1.find('{http://zakupki.gov.ru/oos/types/1}factAddress').text
        list_nodes.append(responsibleorg_factaddress)

        ns_6= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        ns_6_1 = ns_6.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
        responsibleorg_inn = ns_6_1.find('{http://zakupki.gov.ru/oos/types/1}INN').text
        list_nodes.append(responsibleorg_inn)
    
        ns_7= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        ns_7_1 = ns_7.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
        responsibleorg_kpp = ns_7_1.find('{http://zakupki.gov.ru/oos/types/1}KPP').text
        list_nodes.append(responsibleorg_kpp)

        ns_8= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
        responsiblerole = ns_8.find('{http://zakupki.gov.ru/oos/types/1}responsibleRole').text
        list_nodes.append(responsiblerole)
    
        ns_9= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
        placingway_code = ns_9.find('{http://zakupki.gov.ru/oos/types/1}code').text
        list_nodes.append(placingway_code)
    
        ns_10= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
        placingway_name = ns_10.find('{http://zakupki.gov.ru/oos/types/1}name').text
        list_nodes.append(placingway_name)
    
        ns_11= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
        ns_11_1 = ns_11.find('{http://zakupki.gov.ru/oos/types/1}collecting')
        collecting_startdate = ns_11_1.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
        list_nodes.append(collecting_startdate)
    
        ns_12= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
        ns_12_1 = ns_12.find('{http://zakupki.gov.ru/oos/types/1}collecting')
        collecting_place = ns_12_1.find('{http://zakupki.gov.ru/oos/types/1}place').text
        list_nodes.append(collecting_place)
    
        ns_13= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
        ns_13_1 = ns_13.find('{http://zakupki.gov.ru/oos/types/1}collecting')
        collecting_order = ns_13_1.find('{http://zakupki.gov.ru/oos/types/1}order').text
        list_nodes.append(collecting_order)
    
        ns_14= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
        ns_14_1 = ns_14.find('{http://zakupki.gov.ru/oos/types/1}collecting')
        collecting_enddate = ns_14_1.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
        list_nodes.append(collecting_enddate)
        
        try:
            ns_15= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_15_1 = ns_15.find('{http://zakupki.gov.ru/oos/types/1}scoring')
            scoring_date = ns_15_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
            list_nodes.append(scoring_date)
        except: continue
    
        ns_16= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
        ns_16_1 = ns_16.find('{http://zakupki.gov.ru/oos/types/1}bidding')
        bidding_date = ns_16_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
        list_nodes.append(bidding_date)
    
        ns_17= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        max_price = ns_17.find('{http://zakupki.gov.ru/oos/types/1}maxPrice').text
        list_nodes.append(max_price)
    
        ns_18= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        ns_18_1 = ns_18.find('{http://zakupki.gov.ru/oos/types/1}currency')
        currency_code = ns_18_1.find('{http://zakupki.gov.ru/oos/types/1}code').text
        list_nodes.append(currency_code)
    
        ns_19= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        ns_19_1 = ns_19.find('{http://zakupki.gov.ru/oos/types/1}currency')
        currency_name = ns_19_1.find('{http://zakupki.gov.ru/oos/types/1}name').text
        list_nodes.append(currency_name)
    
        try:
            ns_20= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            finance_source = ns_20.find('{http://zakupki.gov.ru/oos/types/1}financeSource').text
            list_nodes.append(finance_source)
        except: continue
    
        ns_21= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        quantity_undefined = ns_21.find('{http://zakupki.gov.ru/oos/types/1}quantityUndefined').text
        list_nodes.append(quantity_undefined)
    
        ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
        ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
        ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
        customer_regnum = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
        list_nodes.append(customer_regnum)
    
        ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
        ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
        ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
        ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
        customer_fullname = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
        list_nodes.append(customer_fullname)
        
        ns23 = element.find('{http://zakupki.gov.ru/oos/types/1}ETP')
        platform_code = ns23.find('{http://zakupki.gov.ru/oos/types/1}code').text
        list_nodes.append(platform_code)
    
    
    csv_writer.writerow(list_nodes)
    

    
xml_data_to_csv.close()
    
    

In [55]:
print(list_head)

['id', 'purchasenumber', 'docpublishdate', 'href', 'printform_url', 'purchaseobjectinfo', 'responsibleorg_regnum', 'responsibleorg_fullname', 'responsibleorg_postaddress', 'responsibleorg_factaddress', 'responsibleorg_inn', 'responsibleorg_kpp', 'responsiblerole', 'placingway_code', 'placingway_name', 'collecting_startdate', 'collecting_place', 'collecting_order', 'collecting_enddate', 'scoring_date', 'bidding_date', 'max_price', 'currency_code', 'currency_name', 'finance_source', 'quantity_undefined', 'customer_regnum', 'customer_fullname', 'platform_code']


Now for 2022

In [58]:
# Now I actually need to extrapolate this for the case when I have multiple xml files. They were in zip folders


import os
from xml.etree import ElementTree as ET
import csv


path = "C:/Users/Имя/termpaper/notifications_2022"

xml_data_to_csv = open('out_notifications_2022.csv', 'w', encoding ='utf-8')
csv_writer = csv.writer(xml_data_to_csv)
    


list_head = []

    

list_head.append('id')
list_head.append('purchasenumber')
list_head.append('docpublishdate')
list_head.append('href')
list_head.append('printform_url')
list_head.append('purchaseobjectinfo')
list_head.append('responsibleorg_regnum')
list_head.append('responsibleorg_fullname')
list_head.append('responsibleorg_postaddress')
list_head.append('responsibleorg_factaddress')
list_head.append('responsibleorg_inn')
list_head.append('responsibleorg_kpp')
list_head.append('responsiblerole')
list_head.append('placingway_code')
list_head.append('placingway_name')
list_head.append('collecting_startdate')
list_head.append('collecting_place')
list_head.append('collecting_order')
list_head.append('collecting_enddate')
list_head.append('scoring_date')
list_head.append('bidding_date')
list_head.append('max_price')
list_head.append('currency_code')
list_head.append('currency_name')
list_head.append('finance_source')
list_head.append('quantity_undefined')
list_head.append('customer_regnum')
list_head.append('customer_fullname')
list_head.append('platform_code')

csv_writer.writerow(list_head)

for filename in os.listdir(path):
    
    # I want to get only xml files since I also have "sig" files
    if not filename.endswith('.xml'): continue
    # And only get files that start with 'epNotificationEF2020_' since these are notifications for electronic auctions
    if not filename.startswith('epNotificationEF2020_'): continue
   
    
    # And now let's parse these files and convert to csv as I did before for just 1 file:
    
    fullname = os.path.join(path, filename)
    
    tree = ET.parse(fullname)
    root = tree.getroot()
    
    for element in root:
        
        list_nodes= []
        
        try:
            iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
            list_nodes.append(iden)
        except:continue
            
        try:
            purchaseNumber = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseNumber').text
            list_nodes.append(purchaseNumber)
        except:continue
       
        try:
            docPublishDate = element.find('{http://zakupki.gov.ru/oos/types/1}docPublishDate').text
            list_nodes.append(docPublishDate)
        except:continue
        
        try:
            href = element.find('{http://zakupki.gov.ru/oos/types/1}href').text
            list_nodes.append(href)
        except: continue
            
        try:
            ns_1= element.find('{http://zakupki.gov.ru/oos/types/1}printForm')
            printform_url = ns_1.find('{http://zakupki.gov.ru/oos/types/1}url')
            list_nodes.append(printform_url.text)
        except: continue
            
        try:
            purchaseObjectInfo = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseObjectInfo').text
            list_nodes.append(purchaseObjectInfo)
        except: continue
            
            
        try:
            ns_2= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_2_1 = ns_2.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_regnum = ns_2_1.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
            list_nodes.append(responsibleorg_regnum)
        except: continue
        
        try:
            ns_3= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_3_1 = ns_3.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_fullname = ns_3_1.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
            list_nodes.append(responsibleorg_fullname)   
        except: continue
            
        try:
            ns_4= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_4_1 = ns_4.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_postadress = ns_4_1.find('{http://zakupki.gov.ru/oos/types/1}postAddress').text
            list_nodes.append(responsibleorg_postadress)
        except: continue
            
        try:
            ns_5= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_5_1 = ns_5.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_factaddress = ns_5_1.find('{http://zakupki.gov.ru/oos/types/1}factAddress').text
            list_nodes.append(responsibleorg_factaddress)
        except: continue
            
        try:
            ns_6= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_6_1 = ns_6.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_inn = ns_6_1.find('{http://zakupki.gov.ru/oos/types/1}INN').text
            list_nodes.append(responsibleorg_inn)
        except: continue
            
        try:
            ns_7= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_7_1 = ns_7.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_kpp = ns_7_1.find('{http://zakupki.gov.ru/oos/types/1}KPP').text
            list_nodes.append(responsibleorg_kpp)
        except: continue
            
        try:
            ns_8= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            responsiblerole = ns_8.find('{http://zakupki.gov.ru/oos/types/1}responsibleRole').text
            list_nodes.append(responsiblerole)
        except: continue
            
            
        try:
            ns_9= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
            placingway_code = ns_9.find('{http://zakupki.gov.ru/oos/types/1}code').text
            list_nodes.append(placingway_code)
        except: continue
            
        try:
            ns_10= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
            placingway_name = ns_10.find('{http://zakupki.gov.ru/oos/types/1}name').text
            list_nodes.append(placingway_name)
        except: continue
            
            
        try:
            ns_11= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_11_1 = ns_11.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_startdate = ns_11_1.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
            list_nodes.append(collecting_startdate)
        except: continue
            
            
        try:
            ns_12= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_12_1 = ns_12.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_place = ns_12_1.find('{http://zakupki.gov.ru/oos/types/1}place').text
            list_nodes.append(collecting_place)
        except: continue 
        try:
            ns_13= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_13_1 = ns_13.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_order = ns_13_1.find('{http://zakupki.gov.ru/oos/types/1}order').text
            list_nodes.append(collecting_order)
        except: continue
            
        try:
            ns_14= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_14_1 = ns_14.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_enddate = ns_14_1.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
            list_nodes.append(collecting_enddate)
        except: continue
        
        try:
            ns_15= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_15_1 = ns_15.find('{http://zakupki.gov.ru/oos/types/1}scoring')
            scoring_date = ns_15_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
            list_nodes.append(scoring_date)
        except: continue
        
        try:
            ns_16= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_16_1 = ns_16.find('{http://zakupki.gov.ru/oos/types/1}bidding')
            bidding_date = ns_16_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
            list_nodes.append(bidding_date)
        except: continue    
            
        try:
            ns_17= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            max_price = ns_17.find('{http://zakupki.gov.ru/oos/types/1}maxPrice').text
            list_nodes.append(max_price)
        except:continue
            
        try:
            ns_18= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_18_1 = ns_18.find('{http://zakupki.gov.ru/oos/types/1}currency')
            currency_code = ns_18_1.find('{http://zakupki.gov.ru/oos/types/1}code').text
            list_nodes.append(currency_code)
        except: continue
            
        try:
            ns_19= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_19_1 = ns_19.find('{http://zakupki.gov.ru/oos/types/1}currency')
            currency_name = ns_19_1.find('{http://zakupki.gov.ru/oos/types/1}name').text
            list_nodes.append(currency_name)
        except:continue
    
        try:
            ns_20= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            finance_source = ns_20.find('{http://zakupki.gov.ru/oos/types/1}financeSource').text
            list_nodes.append(finance_source)
        except: continue
            
        try:
            ns_21= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            quantity_undefined = ns_21.find('{http://zakupki.gov.ru/oos/types/1}quantityUndefined').text
            list_nodes.append(quantity_undefined)
        except: continue
            
        try:
            ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
            ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
            ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
            customer_regnum = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
            list_nodes.append(customer_regnum)
        except: continue
        
        try:
            ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
            ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
            ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
            customer_fullname = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
            list_nodes.append(customer_fullname)
        except: continue
            
        try:
            ns23 = element.find('{http://zakupki.gov.ru/oos/types/1}ETP')
            platform_code = ns23.find('{http://zakupki.gov.ru/oos/types/1}code').text
            list_nodes.append(platform_code)
        except: continue
    
    
    csv_writer.writerow(list_nodes)
    

    
xml_data_to_csv.close()
    

In [40]:
#Notifications for 2020

import os
from xml.etree import ElementTree as ET
import csv


path = "C:/Users/Имя/termpaper/notifications_2020"

xml_data_to_csv = open('out_notifications_2020.csv', 'w', encoding ='utf-8')
csv_writer = csv.writer(xml_data_to_csv)
    


list_head = []

    

list_head.append('id')
list_head.append('purchasenumber')
list_head.append('docpublishdate')
list_head.append('href')
list_head.append('printform_url')
list_head.append('purchaseobjectinfo')
list_head.append('responsibleorg_regnum')
list_head.append('responsibleorg_fullname')
list_head.append('responsibleorg_postaddress')
list_head.append('responsibleorg_factaddress')
list_head.append('responsibleorg_inn')
list_head.append('responsibleorg_kpp')
list_head.append('responsiblerole')
list_head.append('placingway_code')
list_head.append('placingway_name')
list_head.append('collecting_startdate')
list_head.append('collecting_place')
list_head.append('collecting_order')
list_head.append('collecting_enddate')
list_head.append('scoring_date')
list_head.append('bidding_date')
list_head.append('max_price')
list_head.append('currency_code')
list_head.append('currency_name')
list_head.append('finance_source')
list_head.append('quantity_undefined')
list_head.append('customer_regnum')
list_head.append('customer_fullname')
list_head.append('platform_code')

csv_writer.writerow(list_head)

for filename in os.listdir(path):
    
    # I want to get only xml files since I also have "sig" files
    if not filename.endswith('.xml'): continue
    # And only get files that start with 'fksNotificationEA44' since these are notifications for electronic auctions
    if not filename.startswith('fksNotificationEA44'): continue
   
    
    # And now let's parse these files and convert to csv as I did before for just 1 file:
    
    fullname = os.path.join(path, filename)
    
    tree = ET.parse(fullname)
    root = tree.getroot()
    
    for element in root:
        
        list_nodes= []
        
        try:
            iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
            list_nodes.append(iden)
        except:continue
            
        try:
            purchaseNumber = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseNumber').text
            list_nodes.append(purchaseNumber)
        except:continue
       
        try:
            docPublishDate = element.find('{http://zakupki.gov.ru/oos/types/1}docPublishDate').text
            list_nodes.append(docPublishDate)
        except:continue
        
        try:
            href = element.find('{http://zakupki.gov.ru/oos/types/1}href').text
            list_nodes.append(href)
        except: continue
            
        try:
            ns_1= element.find('{http://zakupki.gov.ru/oos/types/1}printForm')
            printform_url = ns_1.find('{http://zakupki.gov.ru/oos/types/1}url')
            list_nodes.append(printform_url.text)
        except: continue
            
        try:
            purchaseObjectInfo = element.find('{http://zakupki.gov.ru/oos/types/1}purchaseObjectInfo').text
            list_nodes.append(purchaseObjectInfo)
        except: continue
            
            
        try:
            ns_2= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_2_1 = ns_2.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_regnum = ns_2_1.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
            list_nodes.append(responsibleorg_regnum)
        except: continue
        
        try:
            ns_3= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_3_1 = ns_3.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_fullname = ns_3_1.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
            list_nodes.append(responsibleorg_fullname)   
        except: continue
            
        try:
            ns_4= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_4_1 = ns_4.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_postadress = ns_4_1.find('{http://zakupki.gov.ru/oos/types/1}postAddress').text
            list_nodes.append(responsibleorg_postadress)
        except: continue
            
        try:
            ns_5= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_5_1 = ns_5.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_factaddress = ns_5_1.find('{http://zakupki.gov.ru/oos/types/1}factAddress').text
            list_nodes.append(responsibleorg_factaddress)
        except: continue
            
        try:
            ns_6= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_6_1 = ns_6.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_inn = ns_6_1.find('{http://zakupki.gov.ru/oos/types/1}INN').text
            list_nodes.append(responsibleorg_inn)
        except: continue
            
        try:
            ns_7= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            ns_7_1 = ns_7.find('{http://zakupki.gov.ru/oos/types/1}responsibleOrg')
            responsibleorg_kpp = ns_7_1.find('{http://zakupki.gov.ru/oos/types/1}KPP').text
            list_nodes.append(responsibleorg_kpp)
        except: continue
            
        try:
            ns_8= element.find('{http://zakupki.gov.ru/oos/types/1}purchaseResponsible')
            responsiblerole = ns_8.find('{http://zakupki.gov.ru/oos/types/1}responsibleRole').text
            list_nodes.append(responsiblerole)
        except: continue
            
            
        try:
            ns_9= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
            placingway_code = ns_9.find('{http://zakupki.gov.ru/oos/types/1}code').text
            list_nodes.append(placingway_code)
        except: continue
            
        try:
            ns_10= element.find('{http://zakupki.gov.ru/oos/types/1}placingWay')
            placingway_name = ns_10.find('{http://zakupki.gov.ru/oos/types/1}name').text
            list_nodes.append(placingway_name)
        except: continue
            
            
        try:
            ns_11= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_11_1 = ns_11.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_startdate = ns_11_1.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
            list_nodes.append(collecting_startdate)
        except: continue
            
            
        try:
            ns_12= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_12_1 = ns_12.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_place = ns_12_1.find('{http://zakupki.gov.ru/oos/types/1}place').text
            list_nodes.append(collecting_place)
        except: continue 
        try:
            ns_13= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_13_1 = ns_13.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_order = ns_13_1.find('{http://zakupki.gov.ru/oos/types/1}order').text
            list_nodes.append(collecting_order)
        except: continue
            
        try:
            ns_14= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_14_1 = ns_14.find('{http://zakupki.gov.ru/oos/types/1}collecting')
            collecting_enddate = ns_14_1.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
            list_nodes.append(collecting_enddate)
        except: continue
        
        try:
            ns_15= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_15_1 = ns_15.find('{http://zakupki.gov.ru/oos/types/1}scoring')
            scoring_date = ns_15_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
            list_nodes.append(scoring_date)
        except: continue
        
        try:
            ns_16= element.find('{http://zakupki.gov.ru/oos/types/1}procedureInfo')
            ns_16_1 = ns_16.find('{http://zakupki.gov.ru/oos/types/1}bidding')
            bidding_date = ns_16_1.find('{http://zakupki.gov.ru/oos/types/1}date').text
            list_nodes.append(bidding_date)
        except: continue    
            
        try:
            ns_17= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            max_price = ns_17.find('{http://zakupki.gov.ru/oos/types/1}maxPrice').text
            list_nodes.append(max_price)
        except:continue
            
        try:
            ns_18= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_18_1 = ns_18.find('{http://zakupki.gov.ru/oos/types/1}currency')
            currency_code = ns_18_1.find('{http://zakupki.gov.ru/oos/types/1}code').text
            list_nodes.append(currency_code)
        except: continue
            
        try:
            ns_19= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_19_1 = ns_19.find('{http://zakupki.gov.ru/oos/types/1}currency')
            currency_name = ns_19_1.find('{http://zakupki.gov.ru/oos/types/1}name').text
            list_nodes.append(currency_name)
        except:continue
    
        try:
            ns_20= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            finance_source = ns_20.find('{http://zakupki.gov.ru/oos/types/1}financeSource').text
            list_nodes.append(finance_source)
        except: continue
            
        try:
            ns_21= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            quantity_undefined = ns_21.find('{http://zakupki.gov.ru/oos/types/1}quantityUndefined').text
            list_nodes.append(quantity_undefined)
        except: continue
            
        try:
            ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
            ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
            ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
            customer_regnum = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}regNum').text
            list_nodes.append(customer_regnum)
        except: continue
        
        try:
            ns_22_0= element.find('{http://zakupki.gov.ru/oos/types/1}lot')
            ns_22= ns_22_0.find('{http://zakupki.gov.ru/oos/types/1}customerRequirements')
            ns_22_1 = ns_22.find('{http://zakupki.gov.ru/oos/types/1}customerRequirement')
            ns_22_2 = ns_22_1.find('{http://zakupki.gov.ru/oos/types/1}customer')
            customer_fullname = ns_22_2.find('{http://zakupki.gov.ru/oos/types/1}fullName').text
            list_nodes.append(customer_fullname)
        except: continue
            
        try:
            ns23 = element.find('{http://zakupki.gov.ru/oos/types/1}ETP')
            platform_code = ns23.find('{http://zakupki.gov.ru/oos/types/1}code').text
            list_nodes.append(platform_code)
        except: continue
    
    
    csv_writer.writerow(list_nodes)
    

    
xml_data_to_csv.close()

Now let's do it for contracts

In [19]:
os.getcwd()

'C:\\Users\\Имя\\termpaper\\examples'

In [20]:
import os
from xml.etree import ElementTree as ET
import csv


path = "C:/Users/Имя/termpaper/contracts"

xml_data_to_csv = open('out_contracts_2021.csv', 'w', encoding ='utf-8')
csv_writer = csv.writer(xml_data_to_csv)
    


list_head = []

list_head.append('id_contract')
list_head.append('publishdate_contract')
list_head.append('purchasenumber')
list_head.append('final_price')
list_head.append('execution_startdate')
list_head.append('execution_enddate')
list_head.append('supplier_inn')


csv_writer.writerow(list_head)

for filename in os.listdir(path):
    
    # I want to get only xml files since I also have "sig" files
    if not filename.endswith('.xml'): continue
    #and I only want contracts, not some additional documents for contracts
    if not filename.startswith('contract_'): continue
    
    # And now let's parse these files and convert to csv as I did before for just 1 file:
    
    fullname = os.path.join(path, filename)
    
    tree = ET.parse(fullname)
    root = tree.getroot()
    
    for element in root:
        
        list_nodes= []
        
        try:
            iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
            list_nodes.append(iden)
        except: continue
        
        try:
            publishdate_contract = element.find('{http://zakupki.gov.ru/oos/types/1}publishDate').text
            list_nodes.append(publishdate_contract)
        except: continue
        
        try:
            ns = element.find('{http://zakupki.gov.ru/oos/types/1}foundation')
            ns_1 = ns.find('{http://zakupki.gov.ru/oos/types/1}fcsOrder')
            ns_2 = ns_1.find('{http://zakupki.gov.ru/oos/types/1}order')
            purchasenumber = ns_2.find('{http://zakupki.gov.ru/oos/types/1}notificationNumber').text
            list_nodes.append(purchasenumber)
        except: continue
            
            
        try:
            ns1 = element.find('{http://zakupki.gov.ru/oos/types/1}priceInfo')
            final_price = ns1.find('{http://zakupki.gov.ru/oos/types/1}price').text
            list_nodes.append(final_price)
        except: continue
            
        try:
            ns2 = element.find('{http://zakupki.gov.ru/oos/types/1}executionPeriod')
            execution_startdate = ns2.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
            list_nodes.append(execution_startdate)
        except: continue
            
            
        try:
            ns3 = element.find('{http://zakupki.gov.ru/oos/types/1}executionPeriod')
            execution_enddate = ns3.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
            list_nodes.append(execution_enddate)
        except: continue
            
        try:
            ns4 = element.find('{http://zakupki.gov.ru/oos/types/1}suppliers')
            ns4_1 = ns4.find('{http://zakupki.gov.ru/oos/types/1}supplier')
            ns4_2 = ns4_1.find('{http://zakupki.gov.ru/oos/types/1}legalEntityRF')
            
            supplier_inn = ns4_2.find('{http://zakupki.gov.ru/oos/types/1}INN').text
            list_nodes.append(supplier_inn)
        except: continue    
    
    csv_writer.writerow(list_nodes)
    

    
xml_data_to_csv.close()

In [52]:
import os
from xml.etree import ElementTree as ET
import csv


path = "C:/Users/Имя/termpaper/contracts_2022"

xml_data_to_csv = open('out_contracts_2022.csv', 'w', encoding ='utf-8')
csv_writer = csv.writer(xml_data_to_csv)
    


list_head = []

list_head.append('id_contract')
list_head.append('publishdate_contract')
list_head.append('purchasenumber')
list_head.append('final_price')
list_head.append('execution_startdate')
list_head.append('execution_enddate')
list_head.append('supplier_inn')


csv_writer.writerow(list_head)

for filename in os.listdir(path):
    
    # I want to get only xml files since I also have "sig" files
    if not filename.endswith('.xml'): continue
    #and I only want contracts, not some additional documents for contracts
    if not filename.startswith('contract_'): continue
    
    # And now let's parse these files and convert to csv as I did before for just 1 file:
    
    fullname = os.path.join(path, filename)
    
    tree = ET.parse(fullname)
    root = tree.getroot()
    
    for element in root:
        
        list_nodes= []
        
        try:
            iden = element.find('{http://zakupki.gov.ru/oos/types/1}id').text
            list_nodes.append(iden)
        except: continue
        
        try:
            publishdate_contract = element.find('{http://zakupki.gov.ru/oos/types/1}publishDate').text
            list_nodes.append(publishdate_contract)
        except: continue
        
        try:
            ns = element.find('{http://zakupki.gov.ru/oos/types/1}foundation')
            ns_1 = ns.find('{http://zakupki.gov.ru/oos/types/1}fcsOrder')
            ns_2 = ns_1.find('{http://zakupki.gov.ru/oos/types/1}order')
            purchasenumber = ns_2.find('{http://zakupki.gov.ru/oos/types/1}notificationNumber').text
            list_nodes.append(purchasenumber)
        except: continue
            
            
        try:
            ns1 = element.find('{http://zakupki.gov.ru/oos/types/1}priceInfo')
            final_price = ns1.find('{http://zakupki.gov.ru/oos/types/1}price').text
            list_nodes.append(final_price)
        except: continue
            
        try:
            ns2 = element.find('{http://zakupki.gov.ru/oos/types/1}executionPeriod')
            execution_startdate = ns2.find('{http://zakupki.gov.ru/oos/types/1}startDate').text
            list_nodes.append(execution_startdate)
        except: continue
            
            
        try:
            ns3 = element.find('{http://zakupki.gov.ru/oos/types/1}executionPeriod')
            execution_enddate = ns3.find('{http://zakupki.gov.ru/oos/types/1}endDate').text
            list_nodes.append(execution_enddate)
        except: continue
            
        try:
            ns4 = element.find('{http://zakupki.gov.ru/oos/types/1}suppliers')
            ns4_1 = ns4.find('{http://zakupki.gov.ru/oos/types/1}supplier')
            ns4_2 = ns4_1.find('{http://zakupki.gov.ru/oos/types/1}legalEntityRF')
            
            supplier_inn = ns4_2.find('{http://zakupki.gov.ru/oos/types/1}INN').text
            list_nodes.append(supplier_inn)
        except: continue    
    
    csv_writer.writerow(list_nodes)
    

    
xml_data_to_csv.close()

# Part 3: Merging the documents together

First, I work with the notification part

In [42]:

import os
import glob
import pandas as pd


In [43]:
extension = 'csv'
os.chdir("C:/Users/Имя/termpaper/examples/notifications_out")
filenames = [i for i in glob.glob('*.{}'.format(extension))] 
print(filenames) #just to check that these are the files I need
notifications_2020_2021 = pd.concat([pd.read_csv(f) for f in filenames ])
#export to csv
notifications_2020_2021.to_csv( "notifications_2020_2021.csv", index=False, encoding='utf-8')

['out_notifications_2020.csv', 'out_notifications_2021_ex.csv']


C:\Users\A773~1\AppData\Local\Temp/ipykernel_27728/215098404.py:5: DtypeWarning: Columns (24,25,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  notifications_2020_2021 = pd.concat([pd.read_csv(f) for f in filenames ])


In [44]:
# notifications 
import pandas as pd
#os.chdir("C:\\notifications_out")
data_notifications = pd.read_csv("notifications_2020_2021.csv", encoding='utf-8') # ex by mistake, it is full ds for not-s
data_notifications

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (24,25,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,purchasenumber,docpublishdate,href,printform_url,purchaseobjectinfo,responsibleorg_regnum,responsibleorg_fullname,responsibleorg_postaddress,responsibleorg_factaddress,...,scoring_date,bidding_date,max_price,currency_code,currency_name,finance_source,quantity_undefined,customer_regnum,customer_fullname,platform_code
0,25277042,101200007320000076,2020-12-03T16:05:12.449+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,"Оказание услуг по химчистке, замене и доставке...",1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,2020-12-14T00:00:00+05:00,2020-12-15T00:00:00+05:00,8.321700e+04,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
1,25285283,101200007320000077,2020-12-04T11:06:56.445+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по очистке кровли здания от сне...,1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,2020-12-15T00:00:00+05:00,2020-12-16T00:00:00+05:00,2.053800e+05,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
2,25307636,101200007320000078,2020-12-07T15:44:40.506+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по техническому обслуживанию и ...,1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,2020-12-16T00:00:00+05:00,2020-12-17T00:00:00+05:00,6.000000e+05,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
3,25235719,111200004620000006,2020-12-01T08:43:22.348+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг подвижной радиотелефонной связи...,1112000046,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ ТАТАР...,"Российская Федерация, 115172, Москва, ПЕР КОТЕ...","Российская Федерация, 115172, Москва, ПЕР КОТЕ...",...,2020-12-10T00:00:00+03:00,2020-12-11T00:00:00+03:00,4.000000e+05,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
4,25235918,111200004620000007,2020-12-01T08:44:09.303+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по Проведению протокольных меро...,1112000046,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ ТАТАР...,"Российская Федерация, 115172, Москва, ПЕР КОТЕ...","Российская Федерация, 115172, Москва, ПЕР КОТЕ...",...,2020-12-10T00:00:00+03:00,2020-12-11T00:00:00+03:00,1.300000e+06,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196551,25978930,2100700004921000008,2021-03-19T08:54:08.382+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по сбору и транспортированию тв...,21007000049,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АЭРО...","127006, Москва г, Долгоруковская ул, дом № 34,...","Российская Федерация, Москва, Маршала Прошляко...",...,2021-03-30T00:00:00+03:00,2021-03-31T00:00:00+03:00,1.344739e+07,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
196552,28062013,2400400000221000001,2021-11-29T22:25:55.635+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,"""Реконструкция специализированного технологиче...",24004000002,"ФЕДЕРАЛЬНОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ ""ЦЕНТРАЛЬНЫЙ...",111116 г. Москва ул. Авиамоторная д.2,"Российская Федерация, 111116, Москва, УЛ. АВИА...",...,2021-12-15T00:00:00+03:00,2021-12-16T00:00:00+03:00,1.233086e+09,RUB,Российский рубль,NaN,NaN,NaN,NaN,NaN
196553,28109234,2400400000221000002,2021-12-01T22:30:59.626+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Услуги по инженерн

Now contracts

In [53]:
#First, let's merge
os.chdir("C:/Users/Имя/termpaper/examples/contracts_out") # in this folder are only my csv contracts for 5 periods
extension = 'csv'
filenames = [i for i in glob.glob('*.{}'.format(extension))] 
print(filenames) #just to check that these are the files I need
contracts_2021_2022 = pd.concat([pd.read_csv(f) for f in filenames ])
#export to csv
contracts_2021_2022.to_csv( "contracts_2021_2022.csv", index=False, encoding='utf-8')

['out_contracts_2021.csv', 'out_contracts_2022.csv']


In [54]:
#Let's import

#os.chdir("C:\\contracts_out")
data_contracts = pd.read_csv("contracts_2021_2022.csv",encoding='utf-8' )
data_contracts

,id_contract,publishdate_contract,purchasenumber,final_price,execution_startdate,execution_enddate,supplier_inn
0,65926119,2021-06-04T15:40:52.634+03:00,NaN,NaN,NaN,NaN,NaN
1,65202701,2021-04-21T15:55:38.762+03:00,1.482000e+17,2.945672e+09,2014-12-28,2021-07-31,5.700000e+09
2,68212160,2021-10-20T15:46:52.641+03:00,1.483000e+17,1.553096e+05,2014-06-01,2015-05-31,7.729666e+09
3,69210940,2021-12-09T15:04:20.124+03:00,NaN,NaN,NaN,NaN,NaN
4,68983634,2021-12-01T11:49:12.833+03:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
548325,70200649,2022-01-12T16:21:21.329+03:00,1.483000e+17,2.210000e+07,2022-01-10,2023-01-31,NaN
548326,70083415,2022-01-10T14:11:26.446+03:00,NaN,NaN,NaN,NaN,NaN
548327,70079630,2022-01-10T11:02:40.173+03:00,NaN,NaN,NaN,NaN,NaN
548328,70045904,2022-01-10T12:32:26.494+03:00,NaN,NaN,NaN,NaN,NaN


In [55]:
data_contracts.shape

(548330, 7)

In [58]:
data_contracts.isna().sum()

id_contract                  0
publishdate_contract         0
purchasenumber          202826
final_price             202987
execution_startdate     202987
execution_enddate       202987
supplier_inn            444646
dtype: int64

As we can see here, some contracts have id number, but miss purchasenumber, final price and all other variables except for id and date. It is because it is data mistakenly parsed from other documents which are additions to the contracts (for contracts which are not finished yet, so not all data is available). I want to disregard these contracts, which I will do in the next steps

In [59]:
data_notifications.shape

(173121, 29)

I want to join these 2 datasets. I will be able to do it by purchasenumber. However, the problem is that if some changes happened in notifications or contracts, they have been updated and new enry has been posted on the ftp server. Let'say say if the contract exceeded the first final price(spending), the new contract will be posted on server. As a result, in both datasets there may be several entries for one purchase number. Based on the date, I need to filter and leave only the relevant one in my datasets (both for notifications and contracts)

In [60]:


#First see that is indeed true and sort data set
test_data = data_notifications.sort_values(by=['purchasenumber', 'docpublishdate'])
test_data.head(25) #even in first 25 entries I have several observations for one purchasenumber but with different dates

# then I want to only keep last observation for each purchasenumber
test_data_2 = test_data.drop_duplicates(subset = ['purchasenumber'], keep = 'last')
test_data_2.head(40)

data_notifications = test_data_2

#Just checking that everything is right
data_notifications.head(25)
data_notifications.purchasenumber.value_counts()

101200007320000076     1
373200063121000681     1
373200063121000673     1
373200063121000674     1
373200063121000675     1
                      ..
373100094321001022     1
373100094321001023     1
373100094321001024     1
373100094321001025     1
2400400000221000004    1
Name: purchasenumber, Length: 173121, dtype: int64

In [61]:
# The same eliminating I need to do for contracts. Again, I only need the last entry for each purchase number
#sort data set
test_data = data_contracts.sort_values(by=['purchasenumber', 'publishdate_contract'])
test_data.head(25) #even in first 25 entries I have several observations for one purchasenumber but with different dates

# then I want to only keep last observation for each purchasenumber
test_data_2 = test_data.drop_duplicates(subset = ['purchasenumber'], keep = 'last')
test_data_2.head(25)


data_contracts = test_data_2

#Just checking that everything is right
data_contracts.head(25)
data_contracts.purchasenumber.value_counts()



1.012000e+17    1
3.732001e+17    1
3.732001e+17    1
3.732001e+17    1
3.732001e+17    1
               ..
3.731000e+17    1
3.731000e+17    1
3.731000e+17    1
3.731000e+17    1
2.400400e+18    1
Name: purchasenumber, Length: 11591, dtype: int64

In [743]:
# Finally I want to join these 2 datasets in one by purchasenumber
# Specifically I want to joint contracts to notifications, so I will use left-join for this
data_final = pd.merge(data_notifications, data_contracts, on='purchasenumber', how='left')

#And just to check that no duplicates appeared after this:
data_final.purchasenumber.value_counts()

101200007320000076     1
373200063121000681     1
373200063121000673     1
373200063121000674     1
373200063121000675     1
                      ..
373100094321001022     1
373100094321001023     1
373100094321001024     1
373100094321001025     1
2400400000221000004    1
Name: purchasenumber, Length: 173121, dtype: int64

In [744]:
data_notifications.shape

(173121, 29)

In [745]:
data_contracts.shape

(11592, 7)

In [746]:
data_final.shape

(173121, 35)

In [747]:
data_final.final_price.isna().sum()

170950

In the end I will have quite a small sample of notifications and contracts which would have been merged together. The reason for this is that procurement contracts are long. And, for example, notifications which were published in a certain year, need several years sometimes for the project to be realised - so the contract most likely will take several years to appear. To account for this, I would need to download much more years of notifications and contracts. I don't have time for this in this short project since downloading even this data took several days. It is a clear limitation

In [748]:
data_final.head()

,id,purchasenumber,docpublishdate,href,printform_url,purchaseobjectinfo,responsibleorg_regnum,responsibleorg_fullname,responsibleorg_postaddress,responsibleorg_factaddress,...,quantity_undefined,customer_regnum,customer_fullname,platform_code,id_contract,publishdate_contract,final_price,execution_startdate,execution_enddate,supplier_inn
0,25277042,101200007320000076,2020-12-03T16:05:12.449+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,"Оказание услуг по химчистке, замене и доставке...",1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25285283,101200007320000077,2020-12-04T11:06:56.445+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по очистке кровли здания от сне...,1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25499274,101200007320000078,2021-01-12T15:23:29.572+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по техническому обслуживанию и ...,1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25707539,101200007321000002,2021-02-16T15:28:24.777+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по организации питания (предста...,1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,25834634,101200007321000003,2021-03-03T13:03:00.404+03:00,https://zakupki.gov.ru/epz/order/notice/ea44/v...,https://zakupki.gov.ru/epz/order/notice/printF...,Оказание услуг по предоставлению срочного неис...,1012000073,ПОЛНОМОЧНОЕ ПРЕДСТАВИТЕЛЬСТВО РЕСПУБЛИКИ БАШКО...,"Российская Федерация, 105066, Москва, УЛИЦА ДО...","Российская Федерация, 105066, Москва, УЛИЦА ДО...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Part 4: Working with variables and feature engineering

In [749]:
# In this final data sets I need to create new variables for procurement outcomes I am interested in
# First, I need execution time which is a difference between execution end date and execution start date

data_final['execution_enddate'] = pd.to_datetime(data_final['execution_enddate'])
data_final['execution_startdate'] = pd.to_datetime(data_final['execution_startdate'], errors = 'coerce')

data_final['execution_period'] = data_final['execution_enddate'] - data_final['execution_startdate']
data_final.execution_period.describe()





count                           2171
mean     320 days 09:05:53.201289728
std      197 days 13:49:16.986626998
min                  0 days 00:00:00
25%                182 days 00:00:00
50%                332 days 00:00:00
75%                396 days 00:00:00
max               2740 days 00:00:00
Name: execution_period, dtype: object

In [750]:
# Second outcome if ratio of final price of the contract (that is final spending, it may be more than agreed in the beginning)
#to reserve price(max_price). Reserve price is published before the auction as a max price the buyer is willing to pay. 
#I am interested in this variable because price itself is a bit measure since all products are diverse
data_final.final_price = data_final.final_price
data_final.max_price = data_final.max_price
data_final['ratio_final_to_reserve_price'] = data_final.final_price / data_final.max_price

In [751]:
# Year and month can also matter, so I want to extract it as well
data_final['docpublishdate_new'] = pd.to_datetime(data_final['docpublishdate'], errors = 'coerce')
data_final['month_year'] = data_final['docpublishdate_new'].dt.to_period('M')

C:\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [752]:
data_final.execution_period.describe()

count                           2171
mean     320 days 09:05:53.201289728
std      197 days 13:49:16.986626998
min                  0 days 00:00:00
25%                182 days 00:00:00
50%                332 days 00:00:00
75%                396 days 00:00:00
max               2740 days 00:00:00
Name: execution_period, dtype: object

In [753]:
#I want to make variable for days which will be numerical

data_final['execution_period'] = data_final['execution_period'].dt.days

In [754]:
data_final.execution_period.describe()

count    2171.000000
mean      320.379088
std       197.575891
min         0.000000
25%       182.000000
50%       332.000000
75%       396.000000
max      2740.000000
Name: execution_period, dtype: float64

# Drop not needed columns

In [755]:
data_final.columns

Index(['id', 'purchasenumber', 'docpublishdate', 'href', 'printform_url',
       'purchaseobjectinfo', 'responsibleorg_regnum',
       'responsibleorg_fullname', 'responsibleorg_postaddress',
       'responsibleorg_factaddress', 'responsibleorg_inn',
       'responsibleorg_kpp', 'responsiblerole', 'placingway_code',
       'placingway_name', 'collecting_startdate', 'collecting_place',
       'collecting_order', 'collecting_enddate', 'scoring_date',
       'bidding_date', 'max_price', 'currency_code', 'currency_name',
       'finance_source', 'quantity_undefined', 'customer_regnum',
       'customer_fullname', 'platform_code', 'id_contract',
       'publishdate_contract', 'final_price', 'execution_startdate',
       'execution_enddate', 'supplier_inn', 'execution_period',
       'ratio_final_to_reserve_price', 'docpublishdate_new', 'month_year'],
      dtype='object')

In [756]:
#Delete those variables I won't use for the NLP later
data_final.drop(['docpublishdate', 'id', 'href', 'printform_url','collecting_startdate', 'collecting_place',
       'collecting_order', 'collecting_enddate', 'scoring_date',
       'bidding_date','currency_code', 'currency_name','platform_code', 'id_contract',
       'publishdate_contract', 'execution_startdate',
       'execution_enddate', ], axis=1, inplace=True)

In [757]:
data_final.columns

Index(['purchasenumber', 'purchaseobjectinfo', 'responsibleorg_regnum',
       'responsibleorg_fullname', 'responsibleorg_postaddress',
       'responsibleorg_factaddress', 'responsibleorg_inn',
       'responsibleorg_kpp', 'responsiblerole', 'placingway_code',
       'placingway_name', 'max_price', 'finance_source', 'quantity_undefined',
       'customer_regnum', 'customer_fullname', 'final_price', 'supplier_inn',
       'execution_period', 'ratio_final_to_reserve_price',
       'docpublishdate_new', 'month_year'],
      dtype='object')

In [758]:
data_final.responsiblerole.describe()

count     173121
unique        12
top           CU
freq      145851
Name: responsiblerole, dtype: object

In [759]:
#drop more not needed columns
data_final.drop(['docpublishdate_new','supplier_inn', 'customer_regnum','customer_fullname', 'responsibleorg_kpp', 'placingway_name','placingway_code','finance_source','quantity_undefined'], axis=1, inplace=True)

In [761]:
data_final.columns

Index(['purchasenumber', 'purchaseobjectinfo', 'responsibleorg_regnum',
       'responsibleorg_fullname', 'responsibleorg_postaddress',
       'responsibleorg_factaddress', 'responsibleorg_inn', 'responsiblerole',
       'max_price', 'final_price', 'execution_period',
       'ratio_final_to_reserve_price', 'month_year'],
      dtype='object')

# Dealing with missing values

As described before, there was a mistake in parsing and I want only to keep observation for which i have final_price. Because for those who don't it means there was no contract final details yet (due to the fact that I downloaded only data for recent year). Thus, I only keep data set with present purchasenumber

In [762]:
data_final.shape

(173121, 13)

In [763]:
data_final.final_price.isna().sum()

170950

In [764]:
data_final = data_final[data_final['final_price'].notna()]

In [765]:
data_final.shape

(2171, 13)

In [766]:
data_final.head

<bound method NDFrame.head of             purchasenumber                                 purchaseobjectinfo  \
17      101200007321000016                 Поставка чистящих и моющих средств   
40      104200002921000016            Поставка салфеток (искусственная замша)   
59      111200004620000016  Оказание услуг по текущему ремонту транспортны...   
76      111200004621000016  Услуги по медицинскому обслуживанию сотруднико...   
114     122100013521000016  Продление срока действия неисключительных прав...   
...                    ...                                                ...   
172541  873500001521000064    Поставка комплектующих для компьютерной техники   
172621  895100000121000064  Оказание услуг по выполнению инженерно-геодези...   
172728  895100000121000192  Поставка (приобретение) бумаги для офисной тех...   
172823  895100000121000320  Оказание\r\r\nуслуг по открытию и ведению (в т...   
172922  900700002121000064  Поставка характериографов полупроводниковых\r\...  

Now with this small sample I will do further analysis. Let's see which other missing values I have

In [767]:
#Function to define table with missing values
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [768]:
missing_zero_values_table(data_final)

Your selected dataframe has 13 columns and 2171 Rows.
There are 2 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
max_price,0,12,0.6,12,0.6,float64
ratio_final_to_reserve_price,0,12,0.6,12,0.6,float64


I am missing a small percent of the prices, I will inpute mean for those missing values

In [769]:
import statistics
data_final['max_price'] = data_final['max_price'].fillna(value=data_final.max_price.mean()) 

In [770]:
data_final['ratio_final_to_reserve_price'] = data_final['ratio_final_to_reserve_price'].fillna(value=data_final.ratio_final_to_reserve_price.mean()) 

In [771]:
#let's check
missing_zero_values_table(data_final) #perfect

Your selected dataframe has 13 columns and 2171 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


# Adding variable for zipcode which determines location

Now after data set is cleaned I can add variable for zipcode

Another variable that may be of interest is the location of the purchase (where the buyer is located). Perhaps it is important because this specific district in Moscow is less/more corrupt or there are some other location specific things that can affect results. Location can be inferred from zip code. And zip code can be extracted from the column responsible for post address of the buyer

In [772]:
data_final.responsibleorg_postaddress.head(10)

17     Российская Федерация, 105066, Москва, УЛИЦА ДО...
40     Российская Федерация, 119004, Москва, УЛ АЛЕКС...
59     Российская Федерация, 115172, Москва, ПЕР КОТЕ...
76     Российская Федерация, 115172, Москва, ПЕР КОТЕ...
114    Российская Федерация, 109544, Москва, УЛИЦА ШК...
144    Российская Федерация, 101000, Москва, ул.Марос...
214    143350, г.Москва, д. Марушкино, ул.Липовая алл...
272    Российская Федерация, 108840, Москва, Троицк г...
298    Российская Федерация, 108840, Москва, Троицк г...
324    Российская Федерация, 108840, Москва, Троицк г...
Name: responsibleorg_postaddress, dtype: object

In [809]:
data_final.responsibleorg_postaddress.astype(str)

17        Российская Федерация, 105066, Москва, УЛИЦА ДО...
40        Российская Федерация, 119004, Москва, УЛ АЛЕКС...
59        Российская Федерация, 115172, Москва, ПЕР КОТЕ...
76        Российская Федерация, 115172, Москва, ПЕР КОТЕ...
114       Российская Федерация, 109544, Москва, УЛИЦА ШК...
                                ...                        
172541    Российская Федерация, 125124, Москва, УЛИЦА РА...
172621    Российская Федерация, 103132, Москва, Ипатьевс...
172728    Российская Федерация, 103132, Москва, Ипатьевс...
172823    Российская Федерация, 103132, Москва, Ипатьевс...
172922    124527, Г МОСКВА, Г. ЗЕЛЕНОГРАД, АЛЛЕЯ СОЛНЕЧН...
Name: responsibleorg_postaddress, Length: 2171, dtype: object

In [823]:
import regex as re
zip_list = []
for address in data_final.responsibleorg_postaddress:
        try:
            if match is not None:
                zip_code = re.match('^.*(?P<zipcode>\d{6}).*$', address).groupdict()['zipcode']
                zip_list.append(zip_code)
        except: 
                zip_list.append('mistake')

In [824]:
len(zip_list)

2171

In [828]:
df_zip_list = pd.DataFrame(zip_list)

In [834]:
df_zip_list.describe()

,0
count,2171
unique,248
top,125284
freq,93


In [836]:
missing_zero_values_table(df_zip_list)

Your selected dataframe has 1 columns and 2171 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


In [832]:
df_zip_list.shape

(2171, 1)

In [833]:
data_final.shape

(2171, 14)

In [837]:
data_final['zip_code'] = df_zip_list.iloc[:,0].values


In [867]:
data_final.zip_code.isna().sum()

0

In [868]:
data_final.columns

Index(['purchasenumber', 'purchaseobjectinfo', 'responsibleorg_regnum',
       'responsibleorg_fullname', 'responsibleorg_postaddress',
       'responsibleorg_factaddress', 'responsibleorg_inn', 'responsiblerole',
       'max_price', 'final_price', 'execution_period',
       'ratio_final_to_reserve_price', 'month_year', 'zip_code'],
      dtype='object')

# Part 5: saving the data to replicate later

In [869]:
#change dir

os.chdir('C:/Users/Имя/termpaper/examples')

In [870]:
data_final.to_csv('data_final.csv', encoding = 'utf-8') # I write this final data set to csv

# Part 6: Implementing NLP for feature engineering and preparation for model building

One of the problem in my data is that it is hard to understand which object is being purchased. There are special codes which deterimine the class of purchase, but they are too generic (unless it is medical supplies, where codes are more specific).For example, one such generic class may be food or construction. It does not provide much info. One of the solutions may be use purchaseobject info and apply NLP to get more information

In [871]:
#let's import all the libraries
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error
from math import sqrt
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Имя\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Имя\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Имя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [872]:
def tokenize(text): #let's define function for tokenization
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian')) #i am using russian stop words since my product description is in Russian
    lemmatizer = WordNetLemmatizer()
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [lemmatizer.lemmatize(w.lower().strip()) for w in tokens]
    return tokens

Now let's split data I have in test and train

In [873]:
data_final.columns

Index(['purchasenumber', 'purchaseobjectinfo', 'responsibleorg_regnum',
       'responsibleorg_fullname', 'responsibleorg_postaddress',
       'responsibleorg_factaddress', 'responsibleorg_inn', 'responsiblerole',
       'max_price', 'final_price', 'execution_period',
       'ratio_final_to_reserve_price', 'month_year', 'zip_code'],
      dtype='object')

Let's split into test and train

In [964]:
X = data_final[['responsibleorg_regnum','responsibleorg_fullname', 'execution_period', 'month_year', 'zip_code','responsiblerole','purchaseobjectinfo']]
Y = data_final['ratio_final_to_reserve_price']



In [965]:
missing_zero_values_table(X)

Your selected dataframe has 7 columns and 2171 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


# Encoding categorical variables

Let's first look which variables are categorical


In [966]:
cat_df = X.select_dtypes(include=['object', 'bool'])
cat_cols = X.select_dtypes(include=['object', 'bool']).columns.values

In [967]:
cat_cols

array(['responsibleorg_fullname', 'zip_code', 'responsiblerole',
       'purchaseobjectinfo'], dtype=object)

With purchase infor and buyer name, I already worked as a text data, in postadress and factadress I was only interested for zip code. So the only variable I plan to encode here is zip code and responsible role. I will encode month-year even if it is not shown here, because it appeared to be a problem later when combining sparse matrixes

In [968]:
X.columns

Index(['responsibleorg_regnum', 'responsibleorg_fullname', 'execution_period',
       'month_year', 'zip_code', 'responsiblerole', 'purchaseobjectinfo'],
      dtype='object')

In [969]:
#Let's use one hot encoder fir zip code
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')


In [970]:
# passing columns 
enc_df_zipe = pd.DataFrame(enc.fit_transform(X[['zip_code','month_year','responsiblerole']]).toarray())
#enc_df_zipe = pd.DataFrame(enc.fit_transform(X['zip_code']).toarray())
# merge with main df on key values
#X = X .join(enc_df_zipe)
#X = ohe.fit_transform(X).toarray()
#X.head

In [971]:
enc_df_zipe.index = X.index

In [977]:
X = X .join(enc_df_zipe)

In [978]:
X.isna().sum()

responsibleorg_regnum      0
responsibleorg_fullname    0
execution_period           0
month_year                 0
zip_code                   0
                          ..
263                        0
264                        0
265                        0
266                        0
267                        0
Length: 275, dtype: int64

In [979]:
X.shape

(2171, 275)

In [981]:
 #Now let's drop  columns from which I created dummies
X.drop(['zip_code','responsiblerole','month_year' ], axis=1, inplace=True)

In [983]:
missing_zero_values_table(X)

Your selected dataframe has 272 columns and 2171 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


# Test/train split and tf-idf

In [984]:
#Let's do test/train split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [985]:
# Confirm the correct split of the data
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1736, 272)
y_train shape: (1736,)
X_test shape: (435, 272)
y_test shape: (435,)


In [986]:
X_train['purchaseobjectinfo'] #look how purchase description looks like

113895    Аукцион в электронной форме на право заключени...
111579    Электронный аукцион на право заключения контра...
96062     Электронный аукцион на поставку наборов для об...
108593    Аукцион в электронной форме на право заключени...
69024     Услуги по техническому обслуживанию и ремонту ...
                                ...                        
144287    Право заключения контракта на поставку материа...
59547     Оказание услуг по предоставлению доступа в сет...
92604     Поставка лекарственных препаратов (Адалимумаб)...
18665                   Поставка измерительного инструмента
82832     Оказание услуг по техническому обслуживанию эл...
Name: purchaseobjectinfo, Length: 1736, dtype: object

Let's create a TF-IDF matrix for purchase info with words or pairs of words that show up at least 5 times, as purchase info will have more words

In [987]:
# TF-IDF vertorization for Item Description
tfidf_ItemDV = TfidfVectorizer(min_df=5, tokenizer = tokenize, ngram_range = (1,2))
tfidf_ItemDV.fit(X_train['purchaseobjectinfo'])

#Transform both X_train and X_test
X_train_ItemDV = tfidf_ItemDV.transform(X_train['purchaseobjectinfo'])
X_test_ItemDV = tfidf_ItemDV.transform(X_test['purchaseobjectinfo'])

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Let's do the same for the buyer full name

In [988]:
# TF-IDF vertorization for Buyer Name
tfidf_SNV = TfidfVectorizer(min_df=4, ngram_range = (1,2))
tfidf_SNV.fit(X_train['responsibleorg_fullname'])

#Transform both X_train and X_test 
X_train_SNV = tfidf_SNV.transform(X_train['responsibleorg_fullname'])
X_test_SNV = tfidf_SNV.transform(X_test['responsibleorg_fullname'])

Now let's drop purchase info and buyer full name from initial X matrixes and combine all X matrixes with relevant features


In [989]:
 #dropping
X_train.drop(['responsibleorg_fullname','purchaseobjectinfo' ], axis=1, inplace=True)
X_test.drop(['responsibleorg_fullname','purchaseobjectinfo' ], axis=1, inplace=True)

C:\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [990]:
X_train.shape

(1736, 270)

In [991]:
X_test.shape

(435, 270)

In [992]:
X_train_ItemDV.shape

(1736, 1163)

In [993]:
X_train_SNV.shape

(1736, 1037)

In [994]:
X_train.isna().sum()

responsibleorg_regnum    0
execution_period         0
0                        0
1                        0
2                        0
                        ..
263                      0
264                      0
265                      0
266                      0
267                      0
Length: 270, dtype: int64

Let's merge matrixes together

In [995]:
from scipy.sparse import hstack
from scipy.sparse import vstack

# merge  matrixes together into X_train_sparse
X_train_sparse = hstack((X_train_ItemDV, X_train_SNV, X_train))
# merge  matrixes into  X_test_sparse
X_test_sparse = hstack((X_test_ItemDV, X_test_SNV, X_test))

# Part 7: Model Building

I am going to try to predict the ratio of final to reserver price based on the parametes discusses above. I predict ratio because price itself is a bad measure due to the fact that procurement products are very diverse. This ratio is a measure of the price that in the end was paid for the contract and the initial price the buyer announced to be willing to pay before the electronic auction was held. It may be important information since we can be able to determine potentially dangerous or suspicious contract already on the stage of publishing notificaiton, when we don't have all the information about the contract (obtained only after the works are concluded)

For this I will use and evaluate several models

# LASSO

In [996]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

params =  {'alpha':[0.01,1,10,50]} #i will try to select optimal parameter for lasso
lasso_grid = GridSearchCV(Lasso(),param_grid=params,n_jobs=-1)
lasso_grid.fit(X_train_sparse, y_train) # fit





C:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1288507769840139.0, tolerance: 15039090086012.14
  model = cd_fast.sparse_enet_coordinate_descent(


GridSearchCV(estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.01, 1, 10, 50]})

In [997]:
lasso_grid.best_estimator_ #let's see what is optimal parameter

Lasso(alpha=50)

In [998]:
Lasso_model_1=lasso_grid.best_estimator_.fit(X_train_sparse,y_train)

C:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1288507769840139.0, tolerance: 15039090086012.14
  model = cd_fast.sparse_enet_coordinate_descent(


In [999]:
y_train_pred = Lasso_model_1.predict(X_train_sparse)
y_pred = Lasso_model_1.predict(X_test_sparse)
from sklearn.metrics import r2_score, mean_squared_error
print ("R-squared Training",r2_score(y_train, y_train_pred))
print ("R-squared Testing",r2_score(y_test, y_pred))
print ("Train RMSE : ",np.sqrt(mean_squared_error(y_train, y_train_pred)))
print ("Test RMSE  : ",np.sqrt(mean_squared_error(y_test, y_pred)))

R-squared Training 0.9984489264355411
R-squared Testing -3962041.4557464654
Train RMSE :  366565.94726973225
Test RMSE  :  29601022.057200737


I can't use for evaluation confusion matrix and classification report, because I have a continuos problem, so I use RMSE and R-squared

In [1000]:
#Let's look how many features LASSO used
coeff_used = np.sum(Lasso_model_1.coef_!=0)

print("number of features used: ", coeff_used)

number of features used:  1085


# Random Forest

In [1007]:
from sklearn.ensemble import RandomForestRegressor 

In [1010]:
param_grid={'bootstrap': [True],  #define parameters through which I will try to do gridsearch to choose best
            'max_depth': [10, 5, 3],
            'max_features': ['auto','sqrt'],
            'min_samples_leaf': [4,8],
            'min_samples_split': [6,10],
            'n_estimators': [100]}
RFRegressor = GridSearchCV(RandomForestRegressor(),param_grid, cv=10,n_jobs=-1)
RFRegressor.fit(X_train_sparse,y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [10, 5, 3],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [4, 8],
                         'min_samples_split': [6, 10], 'n_estimators': [100]})

In [1011]:
#Check best parameters
RFRegressor.best_params_


{'bootstrap': True,
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 8,
 'min_samples_split': 10,
 'n_estimators': 100}

In [1012]:
#Implement with best parameters
RF_model=RFRegressor.best_estimator_
RF_model.fit(X_train_sparse, y_train)

RandomForestRegressor(max_depth=3, max_features='sqrt', min_samples_leaf=8,
                      min_samples_split=10)

In [1013]:
y_train_pred = RF_model.predict(X_train_sparse)
y_test_pred = RF_model.predict(X_test_sparse)
from sklearn.metrics import r2_score, mean_squared_error
#print(LR_model1.score(X_train, y_train))
print ("R-squared Training",r2_score(y_train, y_train_pred))
print ("R-squared Testing",r2_score(y_test, y_test_pred))
print ("Train RMSE : ",np.sqrt(mean_squared_error(y_train, y_train_pred)))
print ("Test RMSE : ",np.sqrt(mean_squared_error(y_test, y_test_pred)))

R-squared Training 0.0824151431770287
R-squared Testing -3459.0987917440584
Train RMSE :  8915773.743635645
Test RMSE :  874764.9903288184


# Part 8: Conclusions

In the end, both of the models don't seem good. RMSE seem to be incredibly high. The explanation may be a small sample. I think if it was closer to reality (several thousands instead of just 2 thousand), the results would be better. However, if I would be forced to choose between LASSO and RF, I would choose LASSO since there the R squared on training is good compared to super low  R squared in RF model. And RMSE is lowere in LASSO as well which is an indicator that between these 2 models we should choose LASSO